In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [8]:
import pktypes
from pktypes import *
from pkhands import *

In [4]:
d=Deck()
#d.populate()
#d.shuffle()

In [5]:
#d.shuffle()
d.print_cards()

As Qc 5h 9c 6s 6c Ad 9h 3h Js Kh Jc 3c 2s Qd Kd Ks 5c 2d 9s Jd 8d Jh 2h Qh Qs 10d 4s Kc 10h 4h 7c 6h 5s 6d 8c 5d Ac Ah 2c 10s 3d 7s 7d 8h 7h 4c 3s 10c 4d 9d 8s 


In [ ]:
d.repopulate()

In [ ]:
d.shuffle()

In [ ]:
c=Card('J','s')

In [ ]:
for c in d.deal(4):
    print(c)

In [61]:
#EVAL HANDS
d.repopulate()
d.shuffle()
plrHnad = [c for c in d.deal(2)]
commCards = [c for c in d.deal(5)]
allCards = plrHnad + commCards
for c in allCards:
    print(c)

As
Qs
3h
2c
7s
5h
9h


In [66]:
getHand(allCards)

(True, [1, 2, 3, 4, 5], 405, 'Straight')

In [65]:
allCards = [Card('J','h'), Card('3','d'), Card('4','c'), Card('5','s'), Card('2','d'), Card('2','s'), Card('A','d')]

In [ ]:
values = [v.value for v in allCards]
suits = [v.suit for v in allCards]
ranks = [v.rank for v in allCards]
valcts = Counter(values).values()
suitcts = Counter(suits).values()

In [ ]:
print(Counter(suits))
print(Counter(values))

In [ ]:
print(values)
print(suits)
print(ranks)
print(valcts)
print(suitcts)

In [ ]:
## Use this convention
#isHandType = (True/False, vals of hand, hand score, str(name of hand))
values = [v.value for v in allCards]
suits = [v.suit for v in allCards]
ranks = [v.rank for v in allCards]
#valcts = Counter(values).values()
#suitcts = Counter(suits).values()

#def getHand(values, suits, ranks):
    
    isFlush = isStraight = isQuads = isFull = isTrips = isTwoPair = isPair = False

    isStraight = checkStraight(values)
    isFlush = checkFlush(values, suits)

    if isStraight[0] and isFlush[0]:
        isStraightFlush = checkStraightFlush(isFlush[1])
        if isStraightFlush[0]:
            return isStraightFlush 

    isQuads = checkQuads(values)
    if isQuads[0]:
        return isQuads      

    if isFlush[0]:
        return isFlush
    
    if isStraight[0]:
        return isStraight
    
    isTrips = checkTrips(values)
    isPairs = checkPairs(values)

    if len(isTrips[1])>1:  # Player has 2 trips, so basically a full house
        tripcards = sorted(isTrips[1])
        trip = tripcards[1]
        pair = tripcards[0]
        isFullHouse = (True, (trip, pair), FULLSCORE+trip+(pair/10), "Full House")
        return isFullHouse

    if isTrips[0] and isPairs[0]:
        trip = isTrips[1][0]
        paircards = sorted(isPairs[1])
        pair = paircards[-1]
        isFullHouse = (True, (trip, pair), FULLSCORE+trip+(pair/10), "Full House")
        return isFullHouse

    if isTrips[0]:
        return isTrips

    if isPairs[0]:
        return isPairs

    else:
        highcard = sorted(values)[-1]
        isHighCard = (True, highcard, highcard, "High card")
        return isHighCard


In [ ]:
getHand(values, suits, ranks)

In [ ]:
values = [9,9,7,6,6,8,10]
suits = ['h','d','d','c','h','s','s']
hand = set(values)
print(hand)
print(Counter(suits))

In [ ]:
PAIRSCORE=100
TWOPAIRSCORE=200
TRIPSCORE=300
STRAIGHTSCORE=400
FLUSHSCORE=500
FULLSCORE=600
QUADSCORE=700
STRFLSCORE=800

def checkStraight(vals):
    hand = set(vals)
    if 14 in hand:
        hand.add(1) # Account for Ace being a low card
    for low in range(10,0,-1):
        straight = set(range(low, low+5))
        if len(straight-hand) < 1:
            isStraight = True
            straightval = list(sorted(straight))
            highcard = low+4
            score = STRAIGHTSCORE+int(highcard)
            return (True, straightval, score, "Straight")
    return (False, None, 0, None)

def checkFlush(vals, suits):
    suit_dict = Counter(suits)
    if 5 in suit_dict.values() or 6 in suit_dict.values() or 7 in suit_dict.values():
        flushvals=[]
        for s, cnt in suit_dict.items():
            if cnt>4:
                flush_suit = s
        for idx in range(len(suits)):
            if suits[idx] == flush_suit:
                flushvals.append(vals[idx])
        flushvals = list(sorted(flushvals))
        highcard = flushvals[-1]
        score=FLUSHSCORE+highcard       
        return (True, flushvals, score, "Flush")
    return (False, None, 0, "Flush")

#Call only if both straight and flush
def checkStraightFlush(flushvals):
    if 14 in flushvals:
        flushvals.append(1) # Treat Ace as a low card as well
    flushset = set(flushvals)
    for low in range(10,0,-1):
        straight = set(range(low, low+5))
        if len(straight-flushset) < 1:
            flushvals = list(sorted(flushvals))
            highcard = low+4
            score = STRFLSCORE+int(highcard)
            return (True, flushvals, score, "Straight Flush")
    return (False, None, 0, "Straight Flush")
        
def checkQuads(vals):
    for card, count in Counter(vals).items():
        if count == 4:
            return (True, card, QUADSCORE+int(card), "Quads")
    return (False, None, 0, "Quads")

def checkTrips(vals):
    val_dict = Counter(vals)
    trips = []
    if 3 in val_dict.values():
        hightrip = 0
        for card, count in val_dict.items():
            if count == 3:
                trips.append(card)
                if card > hightrip: hightrip = card
        return (True, trips, TRIPSCORE+hightrip, "Trips")
    return (False, trips, 0, "Trips")

def checkPairs(vals):
    val_dict = Counter(vals)
    if 2 in val_dict.values():
        score = PAIRSCORE
        pairs = []
        highpair = 0
        for card, count in val_dict.items():
            if count == 2:
                pairs.append(card)
                if card > highpair: highpair = card
        if len(pairs)>1:
            score = TWOPAIRSCORE
        return (True, pairs, score+highpair, "Pair_s")
    return (False, None, 0, "Pair_s")



In [ ]:
allCards = [Card('5','d'), Card('5','d'), Card('5','d'), Card('7','d'), Card('7','s'), Card('7','d'), Card('A','d')]

In [ ]:
vals = [v.value for v in allCards]
suits = [v.suit for v in allCards]
ranks = [v.rank for v in allCards]

In [ ]:
checkStraight(vals)

In [ ]:
checkFlush(vals, suits)

In [ ]:
fvals = checkFlush(vals, suits)

In [ ]:
checkStraightFlush(fvals[1])

In [ ]:
checkQuads(vals)

In [ ]:
checkTrips(vals)

In [ ]:
checkPairs(vals)

In [ ]:
for low in range(10,0,-1):
    print(low)
    straight = set(range(low, low+5))
    print(straight)

In [ ]:
r = ['8','4','10','5','6','A','K']

In [ ]:
sorted(r)

In [ ]:
checkStraight(r)

In [ ]:
joe = Player('sdff', 'Joe', bank=44, stack=333)
fred = Player('sdf2', 'Fred', bank=44, stack=333)
willis = Player('sdf3', 'willis', bank=44, stack=333)
sally = Player('sdf9', 'Sally', bank=44, stack=333)
lucy = Player('sdf8', 'Lucy', bank=44, stack=333)
patty = Player('sdf7', 'Patty', bank=44, stack=333)

In [ ]:
print(joe)

In [ ]:
t = Table(5)

In [ ]:
t.addPlayerToLobby(joe)
t.addPlayerToLobby(fred)
t.addPlayerToLobby(willis)
t.addPlayerToLobby(sally)
t.addPlayerToLobby(lucy)
t.addPlayerToLobby(patty)

In [ ]:
joe.atTable

In [ ]:
for player in t.playersInLobby:
    player.sitDown()

In [ ]:
t.dealCommCards(1)

In [ ]:
joe.leaveTable()

In [ ]:
fred.standUp()

In [ ]:
print(t)

In [ ]:
t.startGame()

In [ ]:
print(t.buttonSeatNum)

In [ ]:
t.seatedPlayers

In [ ]:
t.getNextSeatNum(3)

In [ ]:
t.removePlayerFromTable(willis)

In [ ]:
t.resetRound()

In [ ]:
t.moveBtton()

In [ ]:
t.clearPlayerHands()
t.deal()

In [ ]:
print(list(range(5)))

In [ ]:
#init

#create Table
#While Table Online
    #for player in Table.players where player.seated == True
    #create list of seated players
    #get ante
        #update player investment
        #update pot 
    #deal
    #betting round
        #
    #flop
    #betting round
    #turn
    #betting round
    #river
    
    
    
    
    # Update player status (seated or standing)
    # update if table is online or offline



In [ ]:
d.clear()
d.populate()
d.shuffle()

In [ ]:
d.deal(4)

In [ ]:
f=[3,4,8,6]

In [ ]:
f.pop()

In [ ]:
x, y = input("Enter action and amount: ").split() 
print("Number of boys: ", x) 
print("Number of girls: ", y) 
print()

In [ ]:
i=4
j=7
while i<7:
    while j<30:
        print(i,j)
        i+=1
        j+=1

In [ ]:
import csv

with open('./testsims/sim.csv', newline='') as csvfile:
    data = list(csv.reader(csvfile))

print(data)

In [ ]:
c=tuple([5,6])

In [ ]:
a,b=c

In [ ]:
a

In [ ]:
x=[['Call 0'], ['Raise', ' 10'], ['Fold', ' 0'], ['Raise', ' 100'], ['Call', ' 0'], ['Call', ' 0'], ['Call', ' 0'], ['Call', ' 0'], ['Fold', ' 0']]

In [ ]:
x

In [ ]:
x.pop(0)

In [ ]:
x

In [ ]:
for x in enumerate(values):
    print(x)

In [ ]:
values